In [ ]:
import mlflow
import optuna
import lightning.pytorch as pl
import json
from src.datawork import data_module
from src.neural_network import NN
from lightning.pytorch.callbacks import Callback
from lightning.pytorch.loggers import MLFlowLogger

# override Optuna's default logging to ERROR only
optuna.logging.set_verbosity(optuna.logging.ERROR)

with open("config.json","r") as f:
    configs=json.load(f)

RANDOM_SEED:int=configs["RANDOM STATE"]
EPOCHS:int=configs["EPOCHS"]
TRIALS:int=configs["TRIALS"]
EXPERIMENT_NAME="Nested false"

In [ ]:
def objective(trial):

    with mlflow.start_run(nested=True):

        # We optimize the number of layers, hidden units in each layer, dropout and the learning rate.
        n_layers = trial.suggest_int("n_layers", 1, 3)
        dropout = trial.suggest_float("dropout", 0.2, 0.5)
        learning_rate = trial.suggest_float("learning_rate",1e-5,1e-1)

        output_dims = [
            trial.suggest_int(f"n_units_l{i}", 4, 128, log=True) for i in range(n_layers)
        ]

        # od="_".join(str(x) for x in output_dims)
        # version = f"version_{round(dropout,2)}_{round(lr,2)}_{od}"

        pl.seed_everything(RANDOM_SEED, workers=True) # Setting seed for execution
        data=data_module(batch_size=4,seed=RANDOM_SEED)
        model = NN(dropout, output_dims,learning_rate)

        mlf_logger = MLFlowLogger(experiment_name=EXPERIMENT_NAME) #, tracking_uri="file:./ml-runs")

        trainer = pl.Trainer(
            logger=mlf_logger,
            deterministic=True,
            enable_checkpointing=False,
            max_epochs=EPOCHS,
            default_root_dir="./"
        )
        hyperparameters = dict(n_layers=n_layers, dropout=dropout, output_dims=output_dims, lr=lr)
        trainer.fit(model,data)
        error = trainer.callback_metrics["val_loss"].item()

    return error

In [ ]:
def get_or_create_experiment(experiment_name:str):

    if experiment := mlflow.get_experiment_by_name(experiment_name):
        return experiment.experiment_id
    else:
        return mlflow.create_experiment(experiment_name)

experiment_id = get_or_create_experiment(EXPERIMENT_NAME)
experiment_id

# Set the current active MLflow experiment
mlflow.set_experiment(experiment_id=experiment_id)

In [ ]:
with mlflow.start_run(experiment_id=experiment_id, run_name="Experiment run", nested=False):
    # Initialize the Optuna study
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=TRIALS) #, callbacks=[champion_callback])

In [ ]:
with mlflow.start_run(experiment_id=experiment_id, run_name="Experiment run", nested=True):
    # Initialize the Optuna study
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=TRIALS) #, callbacks=[champion_callback])

    # mlflow.log_params(study.best_params)
    # mlflow.log_metric("Lowest val loss", study.best_value)

    # Log tags
    # mlflow.set_tags(
    #     tags={
    #         "project": "Apple Demand Project",
    #         "optimizer_engine": "optuna",
    #         "model_family": "pytorch lightning",
    #         "feature_set_version": 1,
    #     }
    # )

    # Log a fit model instance
    # model = NN(**study.best_params)
    # artifact_path = "model"

    # d=data_module(batch_size=4,seed=42)
    # d.setup()
    # inp_example=next(iter(d.train_dataset))[0]

    # mlflow.pytorch.log_model(
    #     # xgb_model=model,
    #     artifact_path=artifact_path,
    #     input_example=inp_example,
    #     model_format="pt",
    #     metadata={"model_data_version": 1},
    # )

    # Get the logged model uri so that we can load it from the artifact store
    # model_uri = mlflow.get_artifact_uri(artifact_path)